In [1]:
include("../src/RadialBasisFunction.jl")
include("../src/DiscreteAdjoint.jl")
include("../src/GradientDescentTools.jl")

using LinearAlgebra, Plots
using .RadialBasisFunction
using .DiscreteAdjoint
using .GradientDescentTools

const dir_for_figures = "../results/figures/"


"../results/figures/"

## Adjoint method applied to RBF approximation

Fixed some paramters, we obtain an approximated field solving the steady-state heat equation

In [2]:
a, b = 0, 2π
N = 40
X = range(a,b, length=N)
boundary_idx = [1, N]
X_boundary = X[boundary_idx]
internal_idx = 2:(N-1)
X_internal = X[internal_idx]


u(x) = sin.(x)   # Not known in advance
q(x) = -sin.(x)  # Known in advance (param)

boundary_conditions = u(X_boundary)


φ(r) = r .^ 3  # r = |x-x_i| ≥ 0
ddφ(r) = 6*r   #second derivative d²/dx² (because the differential problem deals with u" = d²u/dx²)


n_points_in_stencil = 7


C = global_matrix(X, φ, ddφ, n_points_in_stencil)
u_approximated = solve_RBF(C, X, internal_idx, boundary_idx, boundary_conditions, q)

38-element Vector{Float64}:
  0.16230630440607596
  0.3184441606794323
  0.46639819882924805
  0.6024453236002578
  0.7229552007955176
  0.8247685909891668
  0.9052437503888441
  0.9622961535265828
  0.9944469507504687
  1.000861661907953
  0.9813722655566661
  0.9364816555570994
  0.8673506134657445
  ⋮
 -0.9364816555570911
 -0.9813722655566596
 -1.000861661907948
 -0.9944469507504649
 -0.96229615352658
 -0.9052437503888425
 -0.8247685909891656
 -0.7229552007955165
 -0.602445323600257
 -0.46639819882924743
 -0.31844416067943265
 -0.16230630440607627

Now we want to find a specific field, $u_{opt}$, that satisfy some properties defined by $J(u)$. $J$ can be thought as a "quality" function if it has to be maximized or as a "cost" if it has to be minimized. Let's define it:

In [3]:
# Note that the optimization is carried out only on internal points (in fact boundary poiints are conditioned)...
C_internal = C[internal_idx, internal_idx]

# Defininig J (obj function) as follows menas ~> We want to reach a designed u_target
u_target(x) = (x .- a) .* (b .- x)
u_target_num = u_target(X_internal)
J(u_num) = norm(u_num - u_target_num)^2

# Define ideal parameters that leads to u_target and error functions on parameters
q_target_num = C_internal*u_target_num + C[internal_idx, boundary_idx] * boundary_conditions
q_error(q_num) = norm(q_num - q_target_num)
q_relative_error(q_num) = q_error(q_num) / norm(q_target_num)


q_relative_error (generic function with 1 method)

Set up everything that is required for the optimization with the adjoint method

In [4]:
# Partial derivative of J (used to compute the total derivative of J respect q)
dJdu(u_num, q_num) = 2*(u_num - u_target_num)
dJdq(u_num, q_num) = zeros(size(q_num))

# Defining the initial values of the constraint with the ones obtained from the RBF approximation
u_RBF = u_approximated
f_RBF = C[internal_idx, boundary_idx] * boundary_conditions
q_RBF = C_internal*u_RBF + f_RBF
constraint = RBFEquation(C_internal, u_RBF, q_RBF, f_RBF)

N_iter::Int64 = 1e4    # Number of optimization cycles
eval_freq::Int64 = 10  # Frequency of evaluation of the objective function

10

To reach our goal we can modify some parameters of the problem. In this case we can modify the value of the function $q$ at each point of the object (so we can decide its shape: thus $q$ itself).
To get the sougth parameters $q$ that leads to the desired field $u_{opt}$ we use different iterative gradient methods in the discrete adjoint optimization:
- Barzilai-Borwein with long steps
- Barzilai-Borwein with short steps
- Standard gradient update with fixed steps 

In [5]:
q_opt_BB_long_num, u_opt_BB_long_num, q_results_BB_long, u_results_BB_long, steps_to_eval = adjoint_opt_BB(dJdu, dJdq, constraint, N_iter, eval_freq, BB_long_step)

q_opt_BB_short_num, u_opt_BB_short_num, q_results_BB_short, u_results_BB_short = adjoint_opt_BB(dJdu, dJdq, constraint, N_iter, eval_freq, BB_short_step)

step_size(∇, n) = 1/norm(∇)
q_opt_grad_fixed_steps_num, u_opt_grad_fixed_steps_num, q_results_grad_fixed_steps, u_results_grad_fixed_steps = adjoint_opt(dJdu, dJdq, constraint, N_iter, eval_freq, step_size)

Optimum was found at iterations 6388, before the end


([-1.2388338035894413, -1.9982422629925298, -2.2394456010367976, -2.1949446895067912, -2.0952845012896972, -1.9923225064123382, -1.9919315975781002, -2.03604593452323, -2.078092655830196, -2.1013258838520743  …  -2.102260100818814, -2.0790570490488895, -2.0354934441851062, -1.9883590704672482, -1.9888968874260415, -2.1035451293521534, -2.1960498753042055, -2.238512052363206, -1.9960606947027475, -1.2337398378320446], [1.0137592286762775, 1.9889646902281741, 2.914546358550524, 3.7831714806493477, 4.594749775346762, 5.351841997280724, 6.056841370999185, 6.710044700220561, 7.310466192730675, 7.857024950770411  …  7.85704507953065, 7.310463421164422, 6.7100003456806165, 6.056779573621542, 5.351843095240173, 4.594852432403491, 3.78324087165425, 2.9145280382709786, 1.9888776021359036, 1.0136671965919775], [-0.16041128085776568 -0.324753524851647 … -1.2386120847097566 -1.2388338035894413; -0.3166679938014729 -0.6509059768949845 … -1.9980420653791011 -1.9982422629925298; … ; 0.3166679938014706

## Performances

In [6]:
println("Barzilai-Borwein long steps")
println("Fitness value: $(J(u_opt_BB_long_num))")
println("Relative error on u: $(norm(u_opt_BB_long_num - u_target_num) / norm(u_target_num))")
println("Relative error on q: $(q_relative_error(q_opt_BB_long_num))")
println()

println("Barzilai-Borwein short steps")
println("Fitness value: $(J(u_opt_BB_short_num))")
println("Relative error on u: $(norm(u_opt_BB_short_num - u_target_num) / norm(u_target_num))")
println("Relative error on q: $(q_relative_error(q_opt_BB_short_num))")
println()

println("Standard gradient fixed steps")
println("Fitness value: $(J(u_opt_grad_fixed_steps_num))")
println("Relative error on u: $(norm(u_opt_grad_fixed_steps_num - u_target_num) / norm(u_target_num))")
println("Relative error on q: $(q_relative_error(q_opt_grad_fixed_steps_num))")

Barzilai-Borwein long steps
Fitness value: 6.775462544444628e-17
Relative error on u: 1.8286818539137083e-10
Relative error on q: 2.5555586106513664e-7

Barzilai-Borwein short steps
Fitness value: 7.400623733315896e-12
Relative error on u: 6.043699228582371e-8
Relative error on q: 8.959538628258105e-5

Standard gradient fixed steps
Fitness value: 4.076926017770504
Relative error on u: 0.0448575031475819
Relative error on q: 0.10315492049339207


## Plots

Computing the value of $J$ at each step of the optimization...

In [7]:
J_results_BB_long = [J(u) for u in eachcol(u_results_BB_long)]
J_results_BB_short = [J(u) for u in eachcol(u_results_BB_short)]
J_results_grad_fixed_steps = [J(u) for u in eachcol(u_results_grad_fixed_steps)];

... and plotting the results

In [8]:
p = plot(steps_to_eval, J_results_BB_long, label="Barzilai-Borwein long steps")
plot!(steps_to_eval, J_results_BB_short, label="Barzilai-Borwein short steps")
plot!(steps_to_eval, J_results_grad_fixed_steps, label="Gradient fixed steps")

title!("J(u) vs. number of iterations")
xlabel!("Iteration")
ylabel!("J")

# To limit the axis
ylims!(0, 8)

savefig(dir_for_figures*"J(u)_vs_iter_gradien_steps_comparison.png")


"/home/kevin/Uni/Tesi/thesis_code/results/figures/J(u)_vs_iter_gradien_steps_comparison.png"

Note that:

- *Standard gradient fixed step*s does not converge
- *Barzila-Borwein long steps* is less stable